# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
import chromadb
from chromadb.utils import embedding_functions
from datetime import datetime
from openai import OpenAI
import os
from tavily import TavilyClient
from typing import Dict

from lib.agents import Agent
from lib.tooling import tool

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
# Need to pass embedding function so dimensions match
# The collection was created with an OpenAI embedding model, which has a dimension size of 1536.
# However, it appears that Chroma uses an embedding model with a dimension size of 384 to embed the query.
# So, if the embedding_function is not passed to get_collection(),  and then
# the query() method is called on the collection, this will result in:
# chromadb.errors.InvalidArgumentError: Collection expecting embedding with dimension of 1536, got 384
# Essentially, the dimensions don't match, and passing embedding_function to get_collections() fixes this.

@tool
def retrieve_game(query: str):
    """     Tool Docstring:
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game """
    results = collection.query(
    query_texts=[query],
    n_results=3,
    include=['documents', 'metadatas']
    )
    return results

In [4]:
# question1 = "I like to drive. What games would I like?"
# question2 = "What is the most recent version of Tetris, and when did it come out?"

In [5]:
# collection.peek() # confirm collection has been successfully retrieved
# answer = retrieve_game(question1)
# answer

#### Evaluate Retrieval Tool

In [ ]:
evaluation_prompt = """
    Your task is to evaluate if the documents are enough to respond to the query. 
    
    OUTPUT FORMAT:
    
    Your answer should consist of two parts:
    1. A boolean value for whether the documents are sufficient to respond to the query.
    2. A detailed explanation of your reasoning
    
    Provide your response as a JSON object in the following format:
    
    {
        "sufficient": "True" | "False",
        "reasoning": " ... "
    }
    """

client = OpenAI()

@tool
def evaluate_retrieval(question: str, retrieved_docs: list[str]):
    """ Tool Docstring:
   Based on the user's question and on the list of retrieved documents, 
   it will analyze the usability of the documents to respond to that question. 
   args: 
   - question: original question from user
   - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
   The result includes:
   - useful: whether the documents are useful to answer the question
   - description: description about the evaluation result """
    
    input = f"""

    QUERY: {question}
    
    DOCUMENTS: {retrieved_docs}
    """

    response = client.responses.create(
        instructions=evaluation_prompt,
        model="gpt-4o-mini",
        input=input
    )
    
    return response.output_text

In [7]:
# import json
# print(evaluate_retrieval(question1, answer['documents']))
# print()

# eval = evaluate_retrieval(question2, answer['documents'])
# eval_dict = json.loads(eval)
# print(eval_dict)
# print(eval_dict["reasoning"])

#### Game Web Search Tool

In [ ]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [9]:
# results = web_search(question1)
# results

### Agent

In [ ]:
tools = [retrieve_game, evaluate_retrieval, web_search]

instructions = f"""
You are an expert on the video game industry.
You can answer multistep questions by sequentially calling functions.
You follow a pattern of of Thought and Action. 
Create a plan of execution: 
- Use Thought to describe your thoughts about the question you have been asked. 
- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.
Combine information from multiple sources if needed.

When you think it's over, return the answer, listing all sources that you used.
If you found the answer with the retrieve_game tool, provide the document used to arrive at your answer.
If you used the web_search tool, list all links used to arrive at your answer.
Never return an answer without listing your sources.

Never try to respond directly if the question needs a tool. 
But if you don't have a tool available, you can respond directly. 
The actions you have are the Tools: {tools}.

"""

agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions=instructions
)


In [ ]:
queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?"
]

session_id = "games"

for q in queries:
    print(f"{session_id}: {q}")
    run_object = agent.invoke(query=q, session_id=session_id)
    print(run_object.get_final_state()["messages"][-1].content)
    print()

games: When Pokémon Gold and Silver was released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Thought: I have retrieved information regarding the release date of Pokémon Gold and Silver. It was released in 1999 for the Game Boy Color. 

Action: I can now provide the final answer.

Pokémon Gold and Silver was released in 1999 for the Game Boy Color.

Sources:
- Retrieved game details on Pokémon Gold and Silver.

games: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Thought: I have found that the first 3D platformer featu

### (Optional) Advanced

In [ ]:
# TODO: Update agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes